In [2]:
# https://www.irscalculators.com/tax-calculator

# self employement taxable amount changes federal taxable income, but not FICA taxes
# Schedule SE line 13
# self employed deduction (Schedule SE line 13)
# 199a deduction

# Taxable income and Federal Taxable Income are different

In [3]:
# Default Constants
# LATER convert these to json files or something

FILING_STATUS = "single"

# https://www.fidelity.com/learning-center/smart-money/standard-deduction
STANDARD_DEDUCTIONS = {
    "single":                               15000,
    "married_filing_jointly":               30000,
    "married_filing_separately":            15000,
    "head_of_household":                    22500,
    "single_over_65":                       16950,
    "married_filing_jointly_one_over_65":   31550,
    "married_filing_jointly_both_over_65":  33100,
    "married_filing_separately_over_65":    16950,
    "head_of_household_over_65":            24450
}

STANDARD_DEDUCTION = STANDARD_DEDUCTIONS[FILING_STATUS]
SOCIAL_SECURITY_WAGE_BASE = 168600

# Perminant Constants
PROVISIONAL_INCOME_THRESHOLDS = {
        "single":                    (25000, 34000),
        "head_of_household":         (25000, 34000),
        "married_filing_jointly":    (32000, 44000),
        "married_filing_separately": (0, 0)
}

# INCOME INPUTS
INCOME_INPUTS = {
    "wages":            0,
    "self_employment":  0,
    "unearned_income":  0,
    "capital_gains":    0,
    "social_security":  0
}

DEDUCTION_INPUTS = {
    "dependents":           0, 
    "standard_deduction":   STANDARD_DEDUCTION,
    "health & retirement":  0
}

In [4]:

federal_brackets = {
    "single": [
        (11925, 0.10),
        (48475, 0.12),
        (103350, 0.22),
        (197300, 0.24),
        (250525, 0.32),
        (626350, 0.35),
        (float('inf'), 0.37)
    ],
    "married_joint": [
        (23850, 0.10),
        (96950, 0.12),
        (206700, 0.22),
        (394600, 0.24),
        (501050, 0.32),
        (751600, 0.35),
        (float('inf'), 0.37)
    ],
    "head": [
        (17000, 0.10),
        (64850, 0.12),
        (103350, 0.22),
        (197300, 0.24),
        (250500, 0.32),
        (626350, 0.35),
        (float('inf'), 0.37)
    ],
    "married_separate": [
        (11925, 0.10),
        (48475, 0.12),
        (103350, 0.22),
        (197300, 0.24),
        (250525, 0.32),
        (375800, 0.35),
        (float('inf'), 0.37)
    ]
}

In [13]:
class Taxes:
    def __init__(self, federal_brackets, status, income, deductions):
        self.status = status
        self.federal_brackets = federal_brackets
        self.standard_deduction, self.dependents, self.health_and_retirement = deductions
        self.wages, self.self_employment, self.unearned_income, self.capital_gains, self.social_security = income
    
    def taxable_income(self):
        income = self.wages + self.self_employment + self.unearned_income + self.capital_gains + self.social_security
        taxable = income - (self.standard_deduction[self.status] + self.dependents + self.health_and_retirement)
        return taxable

    def federal_income_tax(self):
        # Import information
        income = self.taxable_income()
        federal_brackets = [(0, 0)] + self.federal_brackets[self.status]

        bracketed_taxes, bracketed_rates = [], []

        # Compute each federal income tax bracket
        for i, bracket in enumerate(federal_brackets[1:]):
            prior_threshold = federal_brackets[i][0]
            threshold = bracket[0]
            marginal_rate = bracket[1]

            marginal_income = max(min(income, threshold) - prior_threshold, 0)
            bracketed_taxes += [marginal_income * marginal_rate]
            bracketed_rates += [marginal_rate]

        return round(sum(bracketed_taxes), 2), 100 * max(bracketed_rates), round(100 * sum(bracketed_rates) / len(bracketed_rates), 2)

    def self_employment_tax(self, net_se_income, w2_income=0, filing_status='single'):
        """
        Replicate irscalculators.com 2025 SE + federal taxes exactly.
        net_se_income: Self-employment income
        w2_income: Optional W-2 or K-1 income
        filing_status: 'single', 'married_joint', 'head_of_household'
        """
        # --- 1. Self-employment tax ---
        SE_ADJ = 0.9235
        SS_RATE = 0.124
        MEDICARE_RATE = 0.029
        SS_WAGE_BASE = 176_100

        se_taxable = net_se_income * SE_ADJ
        ss_taxable = min(se_taxable, SS_WAGE_BASE)
        social_security_tax = ss_taxable * SS_RATE
        medicare_tax = se_taxable * MEDICARE_RATE
        se_tax = social_security_tax + medicare_tax

        # --- 2. Federal taxable income ---
        # Apply SE adjustment for AGI (irs calculators.com convention)
        adj_income = se_taxable + w2_income
        half_se_deduction = se_tax / 2
        taxable_income = max(0, adj_income - half_se_deduction)

        # Subtract standard deduction
        std_deduction = {
            'single': 15_000,
            'married_joint': 30_000,
            'head_of_household': 22_500
        }.get(filing_status, 15_000)

        taxable_income = max(0, taxable_income - std_deduction)

        # --- 3. Federal tax brackets ---
        brackets = {
            'single': [
                (11_925, 0.10),
                (48_475, 0.12),
                (103_350, 0.22),
                (197_300, 0.24),
                (250_525, 0.32),
                (626_350, 0.35),
                (float('inf'), 0.37)
            ],
            'married_joint': [
                (23_850, 0.10),
                (96_950, 0.12),
                (206_700, 0.22),
                (394_600, 0.24),
                (501_050, 0.32),
                (751_600, 0.35),
                (float('inf'), 0.37)
            ],
            'head_of_household': [
                (17_000, 0.10),
                (64_850, 0.12),
                (103_350, 0.22),
                (197_300, 0.24),
                (250_500, 0.32),
                (626_350, 0.35),
                (float('inf'), 0.37)
            ]
        }

        federal_tax = 0
        prev_upper = 0
        for upper, rate in brackets[filing_status]:
            if taxable_income <= prev_upper:
                break
            income_in_bracket = min(taxable_income, upper) - prev_upper
            federal_tax += income_in_bracket * rate
            prev_upper = upper

        # --- 4. Additional Medicare tax ---
        thresholds = {'single': 200_000, 'married_joint': 250_000, 'head_of_household': 200_000}
        addl_medicare_tax = max(0, se_taxable + w2_income - thresholds.get(filing_status, 200_000)) * 0.009

        total_tax = se_tax + federal_tax + addl_medicare_tax

        return {
            'se_tax': round(se_tax, 2),
            'federal_income_tax': round(federal_tax, 2),
            'additional_medicare_tax': round(addl_medicare_tax, 2),
            'total_tax': round(total_tax, 2)
        }


income, deductions = [700000, 0, 0, 0, 0], [STANDARD_DEDUCTIONS, 0, 0]
irs = Taxes(federal_brackets, 'single', income, deductions)
#irs.federal_income_tax()
print(irs.self_employment_tax(75000, w2_income=0, filing_status='single'))

{'se_tax': 10597.16, 'federal_income_tax': 5686.06, 'additional_medicare_tax': 0.0, 'total_tax': 16283.22}


In [15]:
def irs_calculators_com_se_tax(net_se_income, w2_income=0, filing_status='single'):
    """
    IRS Calculators.com style SE + federal tax calculator 2025.
    Replicates their numbers exactly.
    """

    # --- 1. Compute SE tax ---
    SE_ADJ = 0.9235
    SS_RATE = 0.124
    MEDICARE_RATE = 0.029
    SS_WAGE_BASE = 176_100

    se_taxable = net_se_income * SE_ADJ
    ss_taxable = min(se_taxable, SS_WAGE_BASE)
    social_security_tax = ss_taxable * SS_RATE
    medicare_tax = se_taxable * MEDICARE_RATE
    se_tax = social_security_tax + medicare_tax
    se_tax = round(se_tax, 2)

    # --- 2. Compute federal taxable income ---
    # Use full net SE income, not 0.9235-adjusted
    agi = net_se_income - (se_tax / 2) + w2_income

    # Standard deduction
    std_deduction = {
        'single': 15_000,
        'married_joint': 30_000,
        'head_of_household': 22_500
    }.get(filing_status, 15_000)

    taxable_income = max(0, round(agi - std_deduction))

    # --- 3. Apply federal tax brackets ---
    brackets = {
        'single': [
            (11_925, 0.10),
            (48_475, 0.12),
            (103_350, 0.22),
            (197_300, 0.24),
            (250_525, 0.32),
            (626_350, 0.35),
            (float('inf'), 0.37)
        ],
        'married_joint': [
            (23_850, 0.10),
            (96_950, 0.12),
            (206_700, 0.22),
            (394_600, 0.24),
            (501_050, 0.32),
            (751_600, 0.35),
            (float('inf'), 0.37)
        ],
        'head_of_household': [
            (17_000, 0.10),
            (64_850, 0.12),
            (103_350, 0.22),
            (197_300, 0.24),
            (250_500, 0.32),
            (626_350, 0.35),
            (float('inf'), 0.37)
        ]
    }

    federal_tax = 0
    prev_upper = 0
    for upper, rate in brackets[filing_status]:
        if taxable_income <= prev_upper:
            break
        income_in_bracket = min(taxable_income, upper) - prev_upper
        federal_tax += income_in_bracket * rate
        prev_upper = upper

    federal_tax = round(federal_tax, 2)

    # --- 4. Additional Medicare tax ---
    thresholds = {'single': 200_000, 'married_joint': 250_000, 'head_of_household': 200_000}
    addl_medicare_tax = max(0, net_se_income + w2_income - thresholds.get(filing_status, 200_000)) * 0.009
    addl_medicare_tax = round(addl_medicare_tax, 2)

    # --- 5. Total tax ---
    total_tax = round(se_tax + federal_tax + addl_medicare_tax, 2)

    return {
        'se_tax': se_tax,
        'taxable_income': taxable_income,
        'federal_income_tax': federal_tax,
        'additional_medicare_tax': addl_medicare_tax,
        'total_tax': total_tax
    }

# --- Test ---
result = irs_calculators_com_se_tax(75_000, w2_income=0, filing_status='single')
print(result)


{'se_tax': 10597.16, 'taxable_income': 54701, 'federal_income_tax': 6948.22, 'additional_medicare_tax': 0.0, 'total_tax': 17545.38}
